## Prompt Optimization

Automatically create a prompt that is optimized to the model and evaluation metric.

In [1]:
#%pip install --quiet dspy

In [2]:
#GEMINI="gemini-2.0-flash"
#OPENAI="gpt-4o-mini"
CLAUDE="claude-3-7-sonnet-latest"

import os
from dotenv import load_dotenv
load_dotenv("../keys.env")
#assert os.environ["GEMINI_API_KEY"][:2] == "AI",\
#       "Please specify the GEMINI_API_KEY access token in keys.env file"
assert os.environ["ANTHROPIC_API_KEY"][:2] == "sk",\
       "Please specify the ANTHROPIC_API_KEY access token in keys.env file"
assert os.environ["OPENAI_API_KEY"][:2] == "sk",\
       "Please specify the OPENAI_API_KEY access token in keys.env file"

## Data

For simplicity, we'll hardcode the text of the marketing blurbs for an O'Reilly book.

In [3]:
mldp_text="""
The design patterns in this book capture best practices and solutions to recurring problems in machine learning. The authors, three Google engineers, catalog proven methods to help data scientists tackle common problems throughout the ML process. These design patterns codify the experience of hundreds of experts into straightforward, approachable advice.

In this book, you will find detailed explanations of 30 patterns for data and problem representation, operationalization, repeatability, reproducibility, flexibility, explainability, and fairness. Each pattern includes a description of the problem, a variety of potential solutions, and recommendations for choosing the best technique for your situation.

You'll learn how to:

Identify and mitigate common challenges when training, evaluating, and deploying ML models
Represent data for different ML model types, including embeddings, feature crosses, and more
Choose the right model type for specific problems
Build a robust training loop that uses checkpoints, distribution strategy, and hyperparameter tuning
Deploy scalable ML systems that you can retrain and update to reflect new data
Interpret model predictions for stakeholders and ensure models are treating users fairly
"""

In [4]:
# This is how much space is on the backcover.
TARGET_MAX_LINES = 15
TARGET_MAX_CHARS = 2400
TARGET_NUM_OBJECTIVES = 6

## Extract the pieces of the marketing description

In [5]:
import dspy
lm = dspy.LM(CLAUDE, api_key=os.environ['ANTHROPIC_API_KEY'])
dspy.configure(lm=lm)

In [6]:
from typing import List
class BlurbExtraction(dspy.Signature):
    text: str = dspy.InputField(desc="Text from backcover")
    about_topic: str = dspy.OutputField(desc="Extracted line about why the topic of book is worth learning")
    about_book: str = dspy.OutputField(desc="Extracted line about what book contains")
    target_audience: List[str] = dspy.OutputField(desc="Inferred roles such as Data Engineer, Data Analyst that the book is for")
    learning_objectives: List[str] = dspy.OutputField(desc="Extracted learning objectives")

In [7]:
module = dspy.ChainOfThought(BlurbExtraction)
cover_info = module(text=mldp_text)
cover_info

Prediction(
    reasoning='Let me analyze this backcover text to extract the key information:\n\n1. About topic: The text explains that design patterns in machine learning capture best practices and solutions to recurring problems. These patterns codify expert experience into approachable advice.\n\n2. About book: The book contains detailed explanations of 30 patterns for various aspects of machine learning including data representation, operationalization, repeatability, reproducibility, flexibility, explainability, and fairness.\n\n3. Target audience: While not explicitly stated, this book appears to be targeted at data scientists (mentioned directly), ML engineers, and ML practitioners who are implementing machine learning systems.\n\n4. Learning objectives: The "You\'ll learn how to:" section clearly outlines the learning objectives, which include identifying challenges in ML, representing data for different models, choosing appropriate model types, building robust training loops, 

In [8]:
cover_info.toDict()

{'reasoning': 'Let me analyze this backcover text to extract the key information:\n\n1. About topic: The text explains that design patterns in machine learning capture best practices and solutions to recurring problems. These patterns codify expert experience into approachable advice.\n\n2. About book: The book contains detailed explanations of 30 patterns for various aspects of machine learning including data representation, operationalization, repeatability, reproducibility, flexibility, explainability, and fairness.\n\n3. Target audience: While not explicitly stated, this book appears to be targeted at data scientists (mentioned directly), ML engineers, and ML practitioners who are implementing machine learning systems.\n\n4. Learning objectives: The "You\'ll learn how to:" section clearly outlines the learning objectives, which include identifying challenges in ML, representing data for different models, choosing appropriate model types, building robust training loops, deploying sc

## Improve the marketing description components

In [9]:
## Improve the pieces
from typing import Dict, Any
class BlurbImprovement(dspy.Signature):
    current_cover: Dict[str, Any] = dspy.InputField(descr="Current information on book")
    about_topic: str = dspy.OutputField(desc="More catchy statement why topic is worth learning")
    about_book: str = dspy.OutputField(desc="More appealing (to target audience) description of book contents")
    target_audience: List[str] = dspy.OutputField(desc="more aspirational list of roles. Instead of programmer, say software engineer. Restrict to top 3.")
    learning_objectives: List[str] = dspy.OutputField(desc="Learning objectives rephrased or reordered to be more appealing to target audience. Exactly 6.")    
    
module = dspy.ChainOfThought(BlurbImprovement)
improved_cover = module(current_cover=cover_info.toDict())
improved_cover

Prediction(
    reasoning="I need to make the cover text more compelling and aspirational while maintaining the core information. For the about topic section, I'll create a more catchy statement highlighting why design patterns in ML are worth learning. For the about book section, I'll make the description more appealing to the target audience by emphasizing practical benefits. For the target audience, I'll upgrade the roles to be more aspirational (limiting to 3) and for learning objectives, I'll rephrase them to be more appealing while keeping exactly 6 items.",
    about_topic='Unlock the collective wisdom of hundreds of ML experts through powerful design patterns that transform complex machine learning challenges into manageable solutions. These battle-tested patterns are your shortcut to ML excellence, saving you countless hours of trial and error.',
    about_book='Dive into a comprehensive collection of 30 essential design patterns that will revolutionize your machine learning w

In [10]:
from dataclasses import dataclass

@dataclass
class Blurb:
    about_topic: str
    about_book: str
    target_audience: List[str]
    learning_objectives: List[str]
    
    def __init__(self, d: dict):
        self.about_topic = d['about_topic']
        self.about_book = d['about_book']
        self.target_audience = d['target_audience']
        self.learning_objectives = d['learning_objectives']
    
    def toMarketingCopy(self) -> str:
        nl = '\n'
        return f"""{self.about_topic}
        
{self.about_book}
        
{', '.join(self.target_audience)} will learn how to:
{nl.join(['* ' + objective for objective in self.learning_objectives])}
        """

print(Blurb(improved_cover.toDict()).toMarketingCopy())

Unlock the collective wisdom of hundreds of ML experts through powerful design patterns that transform complex machine learning challenges into manageable solutions. These battle-tested patterns are your shortcut to ML excellence, saving you countless hours of trial and error.
        
Dive into a comprehensive collection of 30 essential design patterns that will revolutionize your machine learning workflow. From sophisticated data representation techniques to cutting-edge approaches for fairness and explainability, this practical guide equips you with the tools to build robust, scalable, and ethical ML systems that deliver real-world impact.
        
Machine Learning Engineers, Data Science Leaders, AI Solution Architects will learn how to:
* Master advanced techniques to overcome critical challenges in ML model training, evaluation, and production deployment
* Transform raw data into powerful representations using embeddings, feature crosses, and other state-of-the-art approaches
* S

## Build an evaluator for the final output.

This is the metric that will be optimized

In [11]:
class BlurbScore(dspy.Signature):
    blurb: Blurb = dspy.InputField()
    topic_score: float = dspy.OutputField(desc="0-1 on how appealing the statement on why topic is worth learning")
    contents_score: float = dspy.OutputField(desc="0-1 on how appealing (to target audience) the book contents are")
    objectives_score: List[float] = dspy.OutputField(desc="0-1 score of how appealing the learning objective is to target audience")

module = dspy.ChainOfThought(BlurbScore)
blurb = Blurb(improved_cover.toDict())
score_pred = module(blurb=blurb)
score_pred

Prediction(
    reasoning='Let me evaluate this book blurb for a machine learning design patterns book:\n\nTopic appeal:\nThe blurb positions ML design patterns as a way to access "collective wisdom" from hundreds of experts, framing them as shortcuts to excellence that save time. This is a compelling value proposition for professionals in the field who want to avoid reinventing the wheel. The promise of transforming "complex challenges into manageable solutions" directly addresses a pain point for ML practitioners. This is highly appealing to the target audience.\n\nContents appeal:\nThe book offers 30 "essential" design patterns covering a comprehensive range of ML needs - from data representation to fairness and explainability. The content appears well-structured and practical, focusing on building "robust, scalable, and ethical ML systems." The mention of "real-world impact" suggests the book is practical rather than theoretical. The breadth of coverage (training, evaluation, deplo

In [12]:
def calc_aggregate_score(blurb: Blurb, p: dspy.Prediction) -> float:   
    result = (p.topic_score * 10 + p.contents_score * 10 + sum(p.objectives_score)) / (20 + len(p.objectives_score))
    ## cut 0.1 for every line beyond MAX_LINES, etc.
    marketing_copy = blurb.toMarketingCopy()
    num_lines = len(marketing_copy.splitlines())
    num_chars = len(marketing_copy)
    num_objectives = len(blurb.learning_objectives)
    if num_lines > TARGET_MAX_LINES:
        result -= 0.1 * (num_lines - TARGET_MAX_LINES)
    if num_chars > TARGET_MAX_CHARS:
        result -= 0.01 * (num_chars - TARGET_MAX_CHARS)
    if num_objectives > TARGET_NUM_OBJECTIVES:
        result -= 0.1 * (num_objectives - TARGET_NUM_OBJECTIVES)
    if result < 0:
        result = 0
    return result

In [13]:
calc_aggregate_score(blurb, score_pred)

0.9173076923076924

## Combine into a module that can be optimized all at once

In [14]:
class BlurbPipeline(dspy.Module):
    def __init__(self):
        self.extract_info = dspy.ChainOfThought(BlurbExtraction)
        self.improve_blurb = dspy.ChainOfThought(BlurbImprovement)
        self.scorer = dspy.ChainOfThought(BlurbScore)
        
    def forward(self, in_text: str) -> (Blurb, float):
        cover_info = self.extract_info(text=in_text)
        improved_cover = self.improve_blurb(current_cover=cover_info.toDict())
        blurb = Blurb(improved_cover.toDict())
        score_pred = self.scorer(blurb=blurb)
        return blurb, calc_aggregate_score(blurb, score_pred)
    
program = BlurbPipeline()
blurb, score = program(in_text=mldp_text)
print(score, '\n', blurb.toMarketingCopy())

0.9173076923076924 
 Unlock the collective wisdom of hundreds of ML experts through powerful design patterns that transform complex machine learning challenges into manageable solutions. These battle-tested patterns are your shortcut to ML excellence, saving you countless hours of trial and error.
        
Dive into a comprehensive collection of 30 essential design patterns that will revolutionize your machine learning workflow. From sophisticated data representation techniques to cutting-edge approaches for fairness and explainability, this practical guide equips you with the tools to build robust, scalable, and ethical ML systems that deliver real-world impact.
        
Machine Learning Engineers, Data Science Leaders, AI Solution Architects will learn how to:
* Master advanced techniques to overcome critical challenges in ML model training, evaluation, and production deployment
* Transform raw data into powerful representations using embeddings, feature crosses, and other state-of-t

## Do it several times and choose the best

This is about the best prompt for this one book

In [15]:
import dspy

def score_reward(args, pred) -> float:
    blurb, score = pred
    print(f"""**** {score}
{blurb.toMarketingCopy()}
******
    """)
    return score

optimized_pipeline = dspy.BestOfN(
    module=BlurbPipeline(),
    N=10,
    reward_fn=score_reward,
    threshold=0.95
)

blurb, score = optimized_pipeline(
    in_text=mldp_text
)
print(score, '\n', blurb.toMarketingCopy())

**** 0.9173076923076924
Unlock the collective wisdom of hundreds of ML experts through powerful design patterns that transform complex machine learning challenges into manageable solutions. These battle-tested patterns are your shortcut to ML excellence, saving you countless hours of trial and error.
        
Dive into a comprehensive collection of 30 essential design patterns that will revolutionize your machine learning workflow. From sophisticated data representation techniques to cutting-edge approaches for fairness and explainability, this practical guide equips you with the tools to build robust, scalable, and ethical ML systems that deliver real-world impact.
        
Machine Learning Engineers, Data Science Leaders, AI Solution Architects will learn how to:
* Master advanced techniques to overcome critical challenges in ML model training, evaluation, and production deployment
* Transform raw data into powerful representations using embeddings, feature crosses, and other state-o

In [16]:
lm.inspect_history(n=3)





[2025-04-28T09:39:27.394374]

System message:

Your input fields are:
1. `text` (str): Text from backcover
Your output fields are:
1. `reasoning` (str)
2. `about_topic` (str): Extracted line about why the topic of book is worth learning
3. `about_book` (str): Extracted line about what book contains
4. `target_audience` (list[str]): Inferred roles such as Data Engineer, Data Analyst that the book is for
5. `learning_objectives` (list[str]): Extracted learning objectives
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## text ## ]]
{text}

[[ ## reasoning ## ]]
{reasoning}

[[ ## about_topic ## ]]
{about_topic}

[[ ## about_book ## ]]
{about_book}

[[ ## target_audience ## ]]
{target_audience}        # note: the value you produce must adhere to the JSON schema: {"type": "array", "items": {"type": "string"}}

[[ ## learning_objectives ## ]]
{learning_objectives}        # note: the value you produce must adhere to the JSON schema: {"

## Train on many more descriptions to create an optimized prompt

In [17]:
with open("blurbs.txt") as ifp:
    blurbs = ifp.read()
blurbs = blurbs.split("***")

# can create labeled also
# dspy.Example(context=context, question=question, answer=answer).with_inputs("context", "question"))
blurbs = [dspy.Example(in_text=b.strip()+"\n").with_inputs("in_text") for b in blurbs]
print(blurbs[2])

Example({'in_text': "Machine learning systems are both complex and unique. Complex because they consist of many different components and involve many different stakeholders. Unique because they're data dependent, with data varying wildly from one use case to the next. In this book, you'll learn a holistic approach to designing ML systems that are reliable, scalable, maintainable, and adaptive to changing environments and business requirements.\n\nAuthor Chip Huyen, co-founder of Claypot AI, considers each design decision--such as how to process and create training data, which features to use, how often to retrain models, and what to monitor--in the context of how it can help your system as a whole achieve its objectives. The iterative framework in this book uses actual case studies backed by ample references.\n\nThis book will help you tackle scenarios such as:\n\nEngineering data and choosing the right metrics to solve a business problem\nAutomating the process for continually develop

In [18]:
import dspy
from dspy.teleprompt import BootstrapFewShot

def evaluate_prediction(example, pred, trace) -> float:
    blurb, score = pred
    return score

optimizer = BootstrapFewShot(metric=evaluate_prediction)
optimized_pipeline = optimizer.compile(BlurbPipeline(), trainset=blurbs)
optimized_pipeline.save("optimized_pipeline", save_program=True)

 44%|████▍     | 4/9 [00:00<00:00, 16.81it/s]

Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.


In [19]:
# Shwos how to avoid cached result ...
lm_nocache =  dspy.LM(CLAUDE, api_key=os.environ['ANTHROPIC_API_KEY'], cache=False)
dspy.configure(lm=lm_nocache)
blurb, score = optimized_pipeline(
    in_text=mldp_text
)
print(score, '\n', blurb.toMarketingCopy())
dspy.configure(lm=lm)

0.9115384615384615 
 Master the hidden language of elite ML practitioners. Design patterns are the secret weapon that separates world-class machine learning systems from amateur projects. Distilled from the collective wisdom of hundreds of Google engineers, these battle-tested approaches solve the critical challenges that determine whether your ML projects succeed or fail in real-world applications.
        
This essential playbook delivers 30 powerful design patterns that transform complex ML challenges into manageable solutions. Each pattern is meticulously documented with practical examples, implementation strategies, and decision frameworks that help you select the optimal approach for your specific scenario. Unlike theoretical textbooks, this guide provides concrete solutions for the entire ML lifecycle—from data representation to deployment, explainability, and ethical considerations—giving you an unfair advantage in building production-grade machine learning systems.
        
Ma

In [20]:
lm_nocache.inspect_history(n=3)





[2025-04-28T09:39:34.580396]

System message:

Your input fields are:
1. `text` (str): Text from backcover
Your output fields are:
1. `reasoning` (str)
2. `about_topic` (str): Extracted line about why the topic of book is worth learning
3. `about_book` (str): Extracted line about what book contains
4. `target_audience` (list[str]): Inferred roles such as Data Engineer, Data Analyst that the book is for
5. `learning_objectives` (list[str]): Extracted learning objectives
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## text ## ]]
{text}

[[ ## reasoning ## ]]
{reasoning}

[[ ## about_topic ## ]]
{about_topic}

[[ ## about_book ## ]]
{about_book}

[[ ## target_audience ## ]]
{target_audience}        # note: the value you produce must adhere to the JSON schema: {"type": "array", "items": {"type": "string"}}

[[ ## learning_objectives ## ]]
{learning_objectives}        # note: the value you produce must adhere to the JSON schema: {"